### Different params for transformer encoder. Increase # of heads.

In [1]:
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import math

from scipy.linalg import expm, hadamard, signm
import numpy as np

import os

import seaborn as sns

import logging
import math

import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.insert(0, '/home/rahulpadmanabhan/Development/ws1/masters_thesis_2/LAWT/src/neuralnet')
# sys.path.insert(0, '/home/rahulpadmanabhan/Development/ws1/masters_thesis_2/LAWT/src/')
import datagenerator
from models import MatrixFunctionTransformer
from common import get_logger

2024-09-10 19:40:15,629 - datagenerator - INFO - Using device: cuda


In [3]:
torch.set_default_dtype(torch.float64)

In [4]:
save_dir = "/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sin/encoder_20240909235033"

In [5]:
logger = get_logger()

In [6]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

2024-09-10 19:40:17,200 - root - INFO - Using device: cuda


# Settings

In [7]:
dim = 5
operation = "sin"

In [12]:
test_samples = 2**15  # 16384
k_values = range(5, 19)
train_samples = [32768*2]
dim = dim

In [13]:
logger.info(f"{train_samples=}, {test_samples=}")

relative_errors = []

2024-09-10 19:43:36,186 - root - INFO - train_samples=[65536], test_samples=32768


In [23]:
train_vals = [3*16384 - 16384]
layer=16
models_lst = [*map(lambda x: os.path.join(save_dir, f"dim_{dim}", f'layers_{layer}', f'{operation}_model_{str(x)}.pth'), train_vals)]
train_lst = [*map(lambda x: os.path.join(save_dir, f"dim_{dim}", "train", f'train_dataset_{str(x)}.pt'), train_vals)]
test_lst = [*map(lambda x: os.path.join(save_dir, f"dim_{dim}", "test", f'test_dataset_{str(x)}.pt'), train_vals)]

logger.info(f"{models_lst=}\n\n{train_lst=}\n\n{test_lst=}")

2024-09-10 19:45:40,108 - root - INFO - models_lst=['/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sin/encoder_20240909235033/dim_5/layers_16/sin_model_32768.pth']

train_lst=['/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sin/encoder_20240909235033/dim_5/train/train_dataset_32768.pt']

test_lst=['/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sin/encoder_20240909235033/dim_5/test/test_dataset_32768.pt']


In [24]:
models_lst[0]

'/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sin/encoder_20240909235033/dim_5/layers_16/sin_model_32768.pth'

In [25]:
dim=5
d_model = dim*dim
# Ensure d_model is divisible by nhead
nhead = dim
d_model = (dim // nhead) * nhead
# Load the saved model
model = MatrixFunctionTransformer(d_model, dim, 16).to(device)
model.load_state_dict(torch.load(models_lst[0]))
logger.info(model)

2024-09-10 19:45:41,828 - root - INFO - MatrixFunctionTransformer(
  (encoder1): TransformerEncoder(
    (layers): ModuleList(
      (0-15): 16 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=5, out_features=5, bias=True)
        )
        (linear1): Linear(in_features=5, out_features=20, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=20, out_features=5, bias=True)
        (norm1): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (encoder2): TransformerEncoder(
    (layers): ModuleList(
      (0-15): 16 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=5, out_features=5, bias=True

In [26]:
# models_lst[]
dim=5
d_model = dim*dim
# Ensure d_model is divisible by nhead
nhead = dim
d_model = (dim // nhead) * nhead
model = MatrixFunctionTransformer(d_model, dim, 16).to(device)
model.load_state_dict(torch.load(models_lst[0]))
model.eval()

MatrixFunctionTransformer(
  (encoder1): TransformerEncoder(
    (layers): ModuleList(
      (0-15): 16 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=5, out_features=5, bias=True)
        )
        (linear1): Linear(in_features=5, out_features=20, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=20, out_features=5, bias=True)
        (norm1): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (encoder2): TransformerEncoder(
    (layers): ModuleList(
      (0-15): 16 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=5, out_features=5, bias=True)
        )
        (linear1): Linear(in

In [33]:
test_ds_path = '/mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sin/encoder_20240909235033/dim_5/layers_16/test/test_dataset_32768.pt'

In [34]:
mu_lst = []
sigma_lst = []
y_main_lst = []
u_shaded_lst = []
l_shaded_lst = []
dim=5

d_model = dim*dim
# Ensure d_model is divisible by nhead
nhead = dim
d_model = (dim // nhead) * nhead
model = MatrixFunctionTransformer(d_model, dim, 16).to(device)
model.load_state_dict(torch.load(models_lst[0]))
model.eval()

# Loading the test dataset
test_dataset = torch.load(test_ds_path)

logger.info(f"Loaded model: {models_lst[0]}")
logger.info(f"{len(test_dataset)=}")

actuals = []
predicted = []

logger.info(f"Starting to predict values in the test_dataset")
with torch.no_grad():
    logger.info(f"{dim=}")
    for x, y in test_dataset:
        x = x.view(-1, dim, dim).to(device).to(torch.float64)
        y = y.view(-1, dim, dim).to(device).to(torch.float64)
        # if dim == 1:
            # predicted.append(model(x.view(-1, 1).to(device).to(torch.float64)))
            # actuals.append(y.view(-1,1).to(device).to(torch.float64))
        # else:
            # predicted.append(model(x.view(1, dim*dim).to(device).to(torch.float64)))
            # actuals.append(y.view(1, dim*dim).to(device).to(torch.float64))
                
        predicted.append(model(x))
        actuals.append(y)
        
    predicted = torch.cat(predicted, 0)
    actuals = torch.cat(actuals, 0)
    


/home/rahulpadmanabhan/Development/envs/venv310/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
2024-09-10 19:50:51,066 - root - INFO - Loaded model: /mnt/wd_2tb/thesis_transformers/experiments/transformer_wo_embedding/sin/encoder_20240909235033/dim_5/layers_16/sin_model_32768.pth
2024-09-10 19:50:51,066 - root - INFO - len(test_dataset)=32768
2024-09-10 19:50:51,067 - root - INFO - Starting to predict values in the test_dataset
2024-09-10 19:50:51,068 - root - INFO - dim=5
2024-09-10 19:56:37,194 - root - INFO - 
predicted.shape=torch.Size([32768, 5, 5])
actuals.shape=torch.Size([32768, 5, 5])
2024-09-10 19:56:37,203 - root - INFO - np_predicted.dtype=dtype('float64'),np_actuals.dtype=dtype('float64')
2024-09-10 19:56:37

In [ ]:
logger.info(f"\n{predicted.shape=}\n{actuals.shape=}")

np_predicted = predicted.view(-1).cpu().numpy()
np_actuals = actuals.view(-1).cpu().numpy()

logger.info(f"{np_predicted.dtype=},{np_actuals.dtype=}")

# mu = np.mean(np.log10(np_predicted))
y_is = np.abs(np_predicted - np_actuals)/(np.abs(np_actuals + 1e-6) ) # did this in a hurry, double check denominator
mu = np.mean(np.log10(y_is))
sigma = (np_predicted.shape[0] -1)**-1 * np.sum((np.log10(y_is) - mu)**2)

y_main = 10**mu 

u_shaded = 10**(mu - sigma)
l_shaded = 10**(mu + sigma)

logger.info(f"{mu=},{sigma=}")
logger.info(f"{y_main=},{u_shaded=},{l_shaded=}")

x_raw = np.repeat(train_vals[0], np_predicted.shape[0])
y_raw = np_actuals

logger.info(f"{x_raw.shape=}, {y_raw.shape=}")

mu_lst.append(mu)
sigma_lst.append(sigma)
y_main_lst.append(y_main)
u_shaded_lst.append(u_shaded)
l_shaded_lst.append(l_shaded)

In [35]:
mu_lst # Relative error

[-0.11825660570541068]

### Observations

Increasing Epochs and heads of attention has a better relative error. Went to only ~55% compared to over 100%

#### Checking an example

In [38]:
predicted[0]

tensor([[-0.5669,  0.0280,  0.1150, -0.3110, -0.3589],
        [-0.5557,  0.0029,  0.0847,  0.0966, -0.4501],
        [-0.1266, -0.0240, -0.0596,  0.1872,  0.5718],
        [-0.1165,  0.0226, -0.0491, -0.1236,  0.5633],
        [ 0.3743,  0.0626, -0.0006, -0.4007, -0.2837]], device='cuda:0')

In [40]:
actuals[0]

tensor([[-0.2439, -1.0375,  1.1604, -0.7223, -1.3062],
        [-0.2895, -0.2425,  0.5196,  0.6788, -0.3176],
        [-0.2343,  0.2411, -0.1000, -0.1480,  1.2491],
        [ 0.0095, -0.0631, -1.2723, -0.8501,  1.0528],
        [ 0.7948,  0.6903, -0.4627, -0.8783, -0.3440]], device='cuda:0')

### Relative Error Calculation

In [68]:
relative_error = torch.mean(torch.abs(torch.norm(predicted - actuals, p='fro', dim=(1,2)) / torch.norm(actuals, p='fro', dim=(1,2))))
logger.info(f"Relative error:{relative_error.item()=}")

2024-09-10 20:16:41,642 - root - INFO - Relative error:relative_error.item()=0.8466582103921876


In [64]:
torch.log10(relative_error)

tensor(-0.0723, device='cuda:0')

In [49]:
10**-0.0719

0.8474225179233051

#### Number of accurate samples with a tolerance of 0.05

In [75]:
# recalculating
relative_error = torch.norm(predicted - actuals, p='fro', dim=(-2,-1)) / torch.norm(actuals, p='fro', dim=(-2,-1))

In [73]:
tolerance = 0.05

In [74]:
# Number of accurate samples
accurate_samples = (relative_error <= tolerance).sum().item()
total_samples = relative_error.size(0)
logger.info(f"Number of accurate samples: {accurate_samples}/{total_samples}")

# Calculate the mean and standard deviation of the relative error
mean_relative_error = relative_error.mean().item()
std_relative_error = relative_error.std().item()
logger.info(f"Mean relative error: {mean_relative_error:.4f}")
logger.info(f"Standard deviation of relative error: {std_relative_error:.4f}")

2024-09-10 20:18:11,725 - root - INFO - Number of accurate samples: 0/32768
2024-09-10 20:18:11,726 - root - INFO - Mean relative error: 0.8467
2024-09-10 20:18:11,727 - root - INFO - Standard deviation of relative error: 0.0635
